In [1]:
import asyncio
from pathlib import Path
import logging
import sys
from typing import List, Optional

# Add project root to Python path if needed
project_root = str(Path().resolve().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

from tqdm import tqdm
import pandas as pd
from FileUtils import FileUtils, OutputFileType

from src.semantic_analyzer import SemanticAnalyzer

# Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
# )
# logger = logging.getLogger(__name__)
from src.nb_helpers.environment import setup_notebook_env, verify_environment




In [2]:
# Set up environment and verify
setup_notebook_env(log_level="WARNING")
verify_environment()



2024-12-12 18:59:26,565 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-12-12 18:59:26,572 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage
Environment Check Results:

Basic Setup:
-----------
✓ Project root in path
✓ FileUtils initialized
✓ .env file loaded

Environment Variables:
---------------------
✓ OPENAI_API_KEY set
✓ ANTHROPIC_API_KEY set

Project Structure:
-----------------
✓ Raw data exists
✓ Processed data exists
✓ Configuration exists
✓ Main config.yaml exists

Environment Status: Ready ✓


True

In [3]:
def display_analysis_summary(results: pd.DataFrame) -> None:
    """Display formatted analysis summary."""
    print("\nAnalysis Results Summary")
    print("=" * 50)

    # Basic stats
    print(f"Total rows processed: {len(results)}")
    if "language" in results.columns:
        print(f"Language: {results['language'].iloc[0]}")
    if "processing_time" in results.columns:
        print(
            f"Average processing time: {results['processing_time'].mean():.2f}s"
        )

    # Results by type
    result_sections = {
        "Keywords": [
            col for col in results.columns if col.startswith("keywords_")
        ],
        "Themes": [col for col in results.columns if col.startswith("themes_")],
        "Categories": [
            col for col in results.columns if col.startswith("categories_")
        ],
    }

    for section_name, columns in result_sections.items():
        if columns:
            print(f"\n{section_name} Results")
            print("-" * 50)

            # Display first 3 rows for each result type
            sample_results = results[columns].head(3)

            # Format each cell for display
            for idx, row in sample_results.iterrows():
                print(f"\nRow {idx + 1}:")
                for col in columns:
                    value = row[col]
                    if pd.notna(value):
                        print(f"  {col.split('_', 1)[1]}: {value}")


async def analyze_excel_content(
    content_file: str = "test_content_en.xlsx",
    parameter_file: str = "parameters_en.xlsx",
    content_column: str = "content",
    analysis_types: Optional[List[str]] = None,
    batch_size: int = 10,
) -> None:
    """Run Excel-based analysis with progress reporting."""
    try:
        # Initialize FileUtils
        file_utils = FileUtils()

        # Verify paths with progress
        with tqdm(total=2, desc="Checking files") as pbar:
            content_path = file_utils.get_data_path("raw") / content_file
            pbar.update(1)

            param_path = file_utils.get_data_path("parameters") / parameter_file
            pbar.update(1)

            if not content_path.exists():
                raise FileNotFoundError(
                    f"Content file not found: {content_path}"
                )
            if not param_path.exists():
                raise FileNotFoundError(
                    f"Parameter file not found: {param_path}"
                )

        print("\nStarting analysis:")
        print(f"Content file: {content_path}")
        print(f"Parameter file: {param_path}")
        print(f"Analysis types: {analysis_types or 'all'}")

        # Create and run analyzer
        analyzer = SemanticAnalyzer.from_excel(
            content_file=content_file,
            parameter_file=parameter_file,
            content_column=content_column,
            file_utils=file_utils,
        )

        # Run analysis with progress
        results = await analyzer.analyze_excel(
            analysis_types=analysis_types,
            batch_size=batch_size,
            save_results=True,
            output_file="analysis_results",
            show_progress=True,
        )

        # Display formatted results
        display_analysis_summary(results)

    except Exception as e:
        logger.error(f"Analysis failed: {e}", exc_info=True)




In [ ]:
# if __name__ == "__main__":
print("Running Excel-based analysis...")

# await run(
await analyze_excel_content(
    content_file="test_content_fi.xlsx",
    parameter_file="parameters_fi.xlsx",
    analysis_types=["keywords", "themes"],
)
    # )


Running Excel-based analysis...
2024-12-12 18:59:26,782 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-12-12 18:59:26,791 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


Checking files: 100%|██████████| 2/2 [00:00<00:00, 1000.19it/s]


Starting analysis:
Content file: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\raw\test_content_fi.xlsx
Parameter file: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_fi.xlsx
Analysis types: ['keywords', 'themes']


2024-12-12 18:59:27,579 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-12-12 18:59:27,585 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage
2024-12-12 18:59:27,681 - FileUtils.core.file_utils - INFO - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


INFO: Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


...


Raw LLM response: {'themes': [{'name': 'Tehokkuus', 'description': 'Tehokkuuden parantaminen prosessiautomaation avulla, mikä viittaa liiketoiminnan ja teknologian yhdistämiseen.', 'confidence': 0.95, 'keywords': ['tehokkuus', 'prosessiautomaatio'], 'domain': 'business/technical', 'parent_theme': None}, {'name': 'Asiakastyytyväisyys', 'description': 'Asiakastyytyväisyyden mittaaminen ja sen positiivinen kehitys, mikä osoittaa asiakaspalvelun parantumista.', 'confidence': 0.9, 'keywords': ['asiakastyytyväisyys', 'kehitys'], 'domain': 'business/technical', 'parent_theme': None}], 'evidence': {'Tehokkuus': [{'text': 'Toiminnan tehokkuus parani prosessiautomaation avulla.', 'relevance': 0.9, 'keywords': ['tehokkuus', 'prosessiautomaatio']}], 'Asiakastyytyväisyys': [{'text': 'Asiakastyytyväisyysmittarit osoittavat positiivista kehitystä.', 'relevance': 0.9, 'keywords': ['asiakastyytyväisyys', 'kehitys']}]}, 'relationships': {'Tehokkuus': ['Asiakastyytyväisyys'], 'Asiakastyytyväisyys': ['Te

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: Theme analysis complete
Analysis Progress: 100%|██████████| 2/2 [00:24<00:00, 12.09s/it]
INFO: Combining results...
INFO: Saving results...
2024-12-12 18:59:54,315 - LocalStorage - INFO - Saved Excel file with sheets: ['Analysis Results']
INFO: Saved Excel file with sheets: ['Analysis Results']



Raw LLM response: {'themes': [{'name': 'Pilvipalvelut', 'description': 'Pilvipalvelut tarjoavat skaalautuvan infrastruktuurin, joka mahdollistaa joustavan ja tehokkaan resurssien hallinnan.', 'confidence': 0.95, 'keywords': ['pilvipalvelu', 'skaalautuva', 'infrastruktuuri'], 'domain': 'technical/business'}, {'name': 'Mikropalveluarkkitehtuuri', 'description': 'Mikropalveluarkkitehtuuri mahdollistaa modulaarisen järjestelmäsuunnittelun, mikä parantaa kehityksen joustavuutta ja ylläpidettävyyttä.', 'confidence': 0.9, 'keywords': ['mikropalveluarkkitehtuuri', 'modulaarinen', 'järjestelmäsuunnittelu'], 'domain': 'technical/business'}, {'name': 'Käyttöönotto', 'description': 'Käyttöönotto viittaa prosessiin, jossa pilvipalvelut ja mikropalveluarkkitehtuuri otetaan käyttöön organisaatiossa.', 'confidence': 0.85, 'keywords': ['käyttöönotto', 'infrastruktuuri', 'skaalautuva'], 'domain': 'technical/business', 'parent_theme': 'Pilvipalvelut'}], 'evidence': {'Pilvipalvelut': [{'text': 'Pilvipalv

INFO: Data saved successfully: {'analysis_results_20241212_185954': 'c:\\Users\\tja\\OneDrive - Rastor-instituutti ry\\Tiedostot\\Rastor-instituutti\\kehittäminen\\analytiikka\\repos\\semantic-text-analyzer\\data\\processed\\analysis_results_20241212_185954.xlsx'}
INFO: Saved results to: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\processed\analysis_results_20241212_185954.xlsx
INFO: Results saved to: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\processed\analysis_results_20241212_185954.xlsx
INFO: Analysis completed in 24.26 seconds



Analysis Results Summary
Total rows processed: 6
Language: fi
Average processing time: 24.19s

Keywords Results
--------------------------------------------------

Row 1:
  id: technical_1
  type: technical
  language: fi
  keyword_scores: 0.95, 0.90, 0.85, 0.80
  keyword_domains: technical, technical, technical, technical

Row 2:
  id: technical_2
  type: technical
  language: fi
  keyword_scores: 0.95, 0.90, 0.90, 0.85
  keyword_domains: technical, technical, technical, technical

Row 3:
  id: technical_3
  type: technical
  language: fi
  keyword_scores: 0.95, 0.90, 0.85, 0.92, 0.88, 0.87
  keyword_domains: technical, technical, technical, technical, technical, technical

Themes Results
--------------------------------------------------

Row 1:
  id: technical_1
  type: technical
  language: fi
  theme_descriptions: Koneoppimismallien koulutus suurilla datajoukolla on keskeinen prosessi, joka mahdollistaa kaavojen tunnistamisen ja analysoinnin.; Neuroverkon arkkitehtuuri, joka koos